In [0]:
# Install required libraries
%pip install xgboost lightgbm catboost scikit-optimize --quiet

# Restart Python kernel to load new packages
dbutils.library.restartPython()

In [0]:
# Import Libraries
import mlflow, mlflow.sklearn, mlflow.xgboost
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, os, warnings
from datetime import datetime
warnings.filterwarnings('ignore')
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from scipy.stats import randint as sp_randint, uniform
from scipy.optimize import minimize
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Config
username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
experiment_name = f"/Users/{username}/air-quality-prediction"
mlflow.set_experiment(experiment_name)
DATA_PATH = 'AirQualityUCI.csv'

# Load and Clean Data
df = pd.read_csv(DATA_PATH, sep=';', decimal=',', na_values=[-200, -200.0])
df = df.dropna(axis=1, how='all')

df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H.%M.%S', errors='coerce')

df = df.drop(columns=['Date', 'Time']).dropna(subset=['DateTime']).sort_values('DateTime')
df = df.set_index('DateTime')

for c in df.select_dtypes(np.number).columns:
    df[c] = df[c].interpolate(method='time', limit_direction='both')
df = df.reset_index().dropna()

# Feature Engineering
target_col = 'CO(GT)'
df = df.sort_values('DateTime').set_index('DateTime')

# Creating Lag Features
lag_periods = [1, 2, 3, 6, 12, 24, 48, 72]
for lag in lag_periods:
    df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)

# Creating Rolling Features
windows = [3, 6, 12, 24, 48, 168]
for w in windows:
    roll = df[target_col].shift(1).rolling(window=w, min_periods=1)
    for stat, func in {'mean': roll.mean(), 'std': roll.std(), 'min': roll.min(), 'max': roll.max()}.items():
        df[f'{target_col}_rolling_{stat}_{w}'] = func

# Creating Rate of Change Features
pollutants = ['PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'NOx(GT)', 'NO2(GT)']

for col in pollutants:
    if col in df.columns:
        df[f'{col}_diff_1h'] = df[col].diff(1)
        df[f'{col}_diff_3h'] = df[col].diff(3)
        df[f'{col}_diff_24h'] = df[col].diff(24)

# Creating Pollutant Interaction Features
for i, p1 in enumerate(pollutants):
    if p1 in df.columns:
        for p2 in pollutants[i+1:]:
            if p2 in df.columns:
                df[f'{p1}_x_{p2}'] = df[p1] * df[p2]
                df[f'{p1}_ratio_{p2}'] = df[p1] / (df[p2] + 1e-8)

# 5) Creating Environmental Interaction Features
if 'T' in df.columns and 'AH' in df.columns:
    df['temp_humidity'] = df['T'] * df['AH']
    df['temp_sq'] = df['T'] ** 2
    df['humidity_sq'] = df['AH'] ** 2

if 'T' in df.columns and 'RH' in df.columns:
    df['temp_rh'] = df['T'] * df['RH']

df = df.reset_index()

# Creating Time Based Features
df['hour'] = df['DateTime'].dt.hour
df['day_of_week'] = df['DateTime'].dt.dayofweek
df['month'] = df['DateTime'].dt.month
df['day_of_month'] = df['DateTime'].dt.day
df['week_of_year'] = df['DateTime'].dt.isocalendar().week

# Creating cyclical encodings
df['hour_sin'] = np.sin(2*np.pi*df['hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['hour']/24)
df['dow_sin'] = np.sin(2*np.pi*df['day_of_week']/7)
df['dow_cos'] = np.cos(2*np.pi*df['day_of_week']/7)
df['month_sin'] = np.sin(2*np.pi*(df['month']-1)/12)
df['month_cos'] = np.cos(2*np.pi*(df['month']-1)/12)

# Creating categorical time features
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_rush_hour'] = df['hour'].isin([7, 8, 9, 17, 18, 19]).astype(int)
df['is_night'] = df['hour'].isin([22, 23, 0, 1, 2, 3, 4, 5]).astype(int)
df['is_winter'] = df['month'].isin([12, 1, 2]).astype(int)
df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)

# Drop rows with NaN created by lagging/diffing
df = df.dropna()

# Build out features
exclude = ['DateTime', target_col]
X = df.drop(columns=exclude)
y = df[target_col]
X = X.select_dtypes(include=[np.number]).copy()
feature_cols = X.columns.tolist()

print(f"Total features: {len(feature_cols)}")
print(f"Dataset size: {len(X)}")

# Data Split
n = len(X)
t1, t2 = int(0.7*n), int(0.85*n)
X_train, y_train = X.iloc[:t1], y.iloc[:t1]
X_valid, y_valid = X.iloc[t1:t2], y.iloc[t1:t2]
X_test, y_test = X.iloc[t2:], y.iloc[t2:]

print(f"Train: {len(X_train)}, Valid: {len(X_valid)}, Test: {len(X_test)}")

# Mean Absolute Percentage Error
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)

# Symmetric Mean Absolute Percentage Error
def smape(y_true, y_pred, eps=1e-8):
    denom = (np.abs(y_true) + np.abs(y_pred) + eps) / 2
    return np.mean(np.abs(y_pred - y_true) / denom) * 100

# Mean Absolute Scaled Error
def mase(y_true, y_pred, y_hist):
    denom = np.mean(np.abs(np.diff(y_hist)))
    return np.mean(np.abs(y_true - y_pred)) / (denom + 1e-8)

# Metrics
def metrics(y_true, y_pred, y_hist):
    return dict(
        rmse=np.sqrt(mean_squared_error(y_true, y_pred)),
        mae=mean_absolute_error(y_true, y_pred),
        r2=r2_score(y_true, y_pred),
        mape=np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-8))) * 100,
        smape=smape(y_true, y_pred),
        mase=mase(y_true, y_pred, y_hist)
    )

def make_ttr(estimator):
    return TransformedTargetRegressor(
        estimator, func=np.log1p, inverse_func=np.expm1
    ) if y_train.min() >= 0 else estimator

# Bayesian Optimization Search
tscv = TimeSeriesSplit(n_splits=7)

def run_bayes_search(name, base_est, search_space, n_iter=50):
    # Bayesian optimization for hyperparameter tuning
    with mlflow.start_run(run_name=f"bayes_{name}"):
        search = BayesSearchCV(
            estimator=make_ttr(base_est),
            search_spaces=search_space,
            n_iter=n_iter,
            cv=tscv,
            scoring="neg_root_mean_squared_error",
            n_jobs=-1,
            random_state=42,
            verbose=0
        )
        search.fit(X_train, y_train)
        best = search.best_estimator_
        
        # Log parameters
        for k, v in search.best_params_.items():
            mlflow.log_param(k, v)
        
        # Validate
        yv = best.predict(X_valid)
        for k, v in metrics(y_valid, yv, y_train).items():
            mlflow.log_metric(f"valid_{k}", v)
        
        mlflow.sklearn.log_model(best, "model")
        print(f"✓ {name:20s} valid RMSE={metrics(y_valid, yv, y_train)['rmse']:.4f}")
        return best

# Model Training with Bayesian Optimization
print("\n" + "="*60)
print("TRAINING MODELS WITH BAYESIAN OPTIMIZATION")
print("="*60 + "\n")

# RandomForest
rf_space = {
    "regressor__n_estimators": Integer(300, 1000),
    "regressor__max_depth": Integer(8, 40),
    "regressor__min_samples_split": Integer(2, 20),
    "regressor__min_samples_leaf": Integer(1, 10),
    "regressor__max_features": Real(0.3, 1.0),
}
best_rf = run_bayes_search("RandomForest", 
    RandomForestRegressor(n_jobs=-1, random_state=42), rf_space, n_iter=40)

# ExtraTrees
et_space = {
    "regressor__n_estimators": Integer(400, 1200),
    "regressor__max_depth": Integer(8, 40),
    "regressor__min_samples_split": Integer(2, 20),
    "regressor__min_samples_leaf": Integer(1, 10),
}
best_et = run_bayes_search("ExtraTrees",
    ExtraTreesRegressor(n_jobs=-1, random_state=42), et_space, n_iter=40)

# GradientBoosting
gb_space = {
    "regressor__n_estimators": Integer(300, 1500),
    "regressor__learning_rate": Real(0.01, 0.2, prior='log-uniform'),
    "regressor__max_depth": Integer(3, 8),
    "regressor__min_samples_split": Integer(2, 20),
    "regressor__subsample": Real(0.6, 1.0),
}
best_gb = run_bayes_search("GradientBoosting",
    GradientBoostingRegressor(random_state=42), gb_space, n_iter=40)

# LightGBM
lgb_space = {
    "regressor__n_estimators": Integer(500, 2000),
    "regressor__max_depth": Integer(4, 15),
    "regressor__learning_rate": Real(0.01, 0.2, prior='log-uniform'),
    "regressor__num_leaves": Integer(20, 200),
    "regressor__min_child_samples": Integer(10, 100),
    "regressor__subsample": Real(0.6, 1.0),
    "regressor__colsample_bytree": Real(0.6, 1.0),
    "regressor__reg_alpha": Real(0, 0.5),
    "regressor__reg_lambda": Real(0, 0.5),
}
best_lgb = run_bayes_search("LightGBM",
    lgb.LGBMRegressor(n_jobs=-1, random_state=42, verbosity=-1), lgb_space, n_iter=50)

# CatBoost
cat_space = {
    "regressor__iterations": Integer(500, 2000),
    "regressor__depth": Integer(4, 10),
    "regressor__learning_rate": Real(0.01, 0.2, prior='log-uniform'),
    "regressor__l2_leaf_reg": Real(1, 10),
    "regressor__border_count": Integer(32, 255),
}
best_cat = run_bayes_search("CatBoost",
    CatBoostRegressor(random_state=42, verbose=0), cat_space, n_iter=50)

# XGBoost with early stopping
print("\nTraining XGBoost with early stopping...")
from mlflow.models.signature import infer_signature

with mlflow.start_run(run_name="bayes_XGBoost"):
    # Bayesian search for hyperparameters
    xgb_space = {
        "n_estimators": Integer(500, 2500),
        "max_depth": Integer(3, 12),
        "learning_rate": Real(0.01, 0.3, prior='log-uniform'),
        "subsample": Real(0.6, 1.0),
        "colsample_bytree": Real(0.5, 1.0),
        "min_child_weight": Integer(1, 20),
        "gamma": Real(0, 0.5),
        "reg_alpha": Real(0, 1.0),
        "reg_lambda": Real(0, 2.0),
    }
    
    base_xgb = xgb.XGBRegressor(tree_method="hist", objective="reg:squarederror", random_state=42)
    search = BayesSearchCV(
        base_xgb, xgb_space, n_iter=50, cv=tscv,
        scoring="neg_root_mean_squared_error", n_jobs=-1, random_state=42
    )
    search.fit(X_train, y_train)
    best_p = search.best_params_
    
    for k, v in best_p.items():
        mlflow.log_param(k, v)
    
    # Early stopping with DMatrix
    use_log = float(y_train.min()) >= 0.0
    y_tr = np.log1p(y_train).values if use_log else y_train.values
    y_va = np.log1p(y_valid).values if use_log else y_valid.values
    
    dtrain = xgb.DMatrix(X_train, label=y_tr)
    dvalid = xgb.DMatrix(X_valid, label=y_va)
    
    train_params = {
        "objective": "reg:squarederror",
        "max_depth": int(best_p["max_depth"]),
        "eta": float(best_p["learning_rate"]),
        "subsample": float(best_p["subsample"]),
        "colsample_bytree": float(best_p["colsample_bytree"]),
        "min_child_weight": float(best_p["min_child_weight"]),
        "gamma": float(best_p.get("gamma", 0.0)),
        "alpha": float(best_p.get("reg_alpha", 0.0)),
        "lambda": float(best_p.get("reg_lambda", 1.0)),
        "tree_method": "hist",
        "seed": 42,
    }
    
    booster = xgb.train(
        params=train_params,
        dtrain=dtrain,
        num_boost_round=int(best_p.get("n_estimators", 1500)),
        evals=[(dvalid, "valid")],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # Get best iteration
    best_n = None
    for attr in ("best_ntree_limit", "best_iteration"):
        if hasattr(booster, attr) and getattr(booster, attr) is not None:
            val = getattr(booster, attr)
            best_n = int(val if "ntree" in attr else (val + 1))
            break
    if best_n is None:
        best_n = int(best_p.get("n_estimators", 1000))
    
    # Final model on train+valid
    X_all = pd.concat([X_train, X_valid], axis=0)
    y_all = pd.concat([y_train, y_valid], axis=0)
    
    final_xgb = xgb.XGBRegressor(
        tree_method="hist", objective="reg:squarederror", random_state=42,
        **{**best_p, "n_estimators": best_n}
    )
    best_xgb = (TransformedTargetRegressor(final_xgb, func=np.log1p, inverse_func=np.expm1)
                if use_log else final_xgb)
    best_xgb.fit(X_all, y_all)
    
    yv = best_xgb.predict(X_valid)
    val_m = metrics(y_valid, yv, y_train)
    for k, v in val_m.items():
        mlflow.log_metric(f"valid_{k}", float(v))
    
    signature = infer_signature(X_test, best_xgb.predict(X_test))
    mlflow.sklearn.log_model(best_xgb, "model", signature=signature)
    print(f"✓ {'XGBoost':20s} valid RMSE={val_m['rmse']:.4f}")

# Weighted Ensemble
print("\n" + "="*60)
print("CREATING OPTIMIZED WEIGHTED ENSEMBLE")
print("="*60 + "\n")

# Get validation predictions from all models
models_dict = {
    'RF': best_rf,
    'ET': best_et,
    'GB': best_gb,
    'XGB': best_xgb,
    'LGB': best_lgb,
    'CAT': best_cat
}

val_preds = [model.predict(X_valid) for model in models_dict.values()]

def weighted_predictions(weights, predictions):
    return sum(w * p for w, p in zip(weights, predictions))

def objective(weights, predictions, y_true):
    weighted = weighted_predictions(weights, predictions)
    return mean_squared_error(y_true, weighted)

# Optimize weights
init_weights = np.ones(len(val_preds)) / len(val_preds)
bounds = [(0, 1)] * len(val_preds)
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

result = minimize(
    objective, init_weights,
    args=(val_preds, y_valid),
    bounds=bounds,
    constraints=constraints,
    method='SLSQP'
)

optimal_weights = result.x
print("\nOptimal Ensemble Weights:")
for name, weight in zip(models_dict.keys(), optimal_weights):
    print(f"  {name:10s}: {weight:.4f}")

# Stacking Ensemble
print("\nTraining Stacking Ensemble...")
stack = make_ttr(StackingRegressor(
    estimators=list(models_dict.items()),
    final_estimator=Ridge(alpha=1.0, random_state=42),
    n_jobs=-1
))
stack.fit(X_train, y_train)

# Final Evaluation
print("\n" + "="*60)
print("FINAL TEST SET EVALUATION")
print("="*60 + "\n")

def evaluate(name, model, is_weighted=False, weights=None):
    with mlflow.start_run(run_name=f"final_{name}"):
        if is_weighted:
            # Weighted ensemble prediction
            test_preds = [m.predict(X_test) for m in models_dict.values()]
            ypred = weighted_predictions(weights, test_preds)
        else:
            ypred = model.predict(X_test)
        
        m = metrics(y_test, ypred, y_train)
        for k, v in m.items():
            mlflow.log_metric(f"test_{k}", float(v))
        
        if not is_weighted:
            signature = infer_signature(X_test, ypred)
            mlflow.sklearn.log_model(model, "model", signature=signature)
        
        print(f"{name:20s}: RMSE={m['rmse']:.4f}, MAE={m['mae']:.4f}, R²={m['r2']:.4f}, SMAPE={m['smape']:.3f}%")
        return m

# Evaluate all models
results = {}
results['RandomForest'] = evaluate("RandomForest", best_rf)
results['ExtraTrees'] = evaluate("ExtraTrees", best_et)
results['GradientBoosting'] = evaluate("GradientBoosting", best_gb)
results['XGBoost'] = evaluate("XGBoost", best_xgb)
results['LightGBM'] = evaluate("LightGBM", best_lgb)
results['CatBoost'] = evaluate("CatBoost", best_cat)
results['StackEnsemble'] = evaluate("StackEnsemble", stack)
results['WeightedEnsemble'] = evaluate("WeightedEnsemble", None, 
                                       is_weighted=True, weights=optimal_weights)

# Model Comparision
print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60 + "\n")

comp_df = pd.DataFrame(results).T.sort_values('rmse')
display(comp_df)

# Find best model
best_name = comp_df.index[0]
print(f"\nBEST MODEL: {best_name}")
print(f"   RMSE: {comp_df.loc[best_name, 'rmse']:.4f}")
print(f"   R²:   {comp_df.loc[best_name, 'r2']:.4f}")
print(f"   SMAPE: {comp_df.loc[best_name, 'smape']:.3f}%")

---------------------------------------------------------------------------
InvalidParameterError                     Traceback (most recent call last)
File <command-8103110435742142>, line 478
    475     importance_title = best_name
    477 print(f"\nCalculating permutation importance for {importance_title}...")
--> 478 pi = permutation_importance(
    479     importance_model, X_test.iloc[:2000], y_test.iloc[:2000],
    480     n_repeats=10, random_state=42, n_jobs=-1
    481 )
    483 imp_df = pd.DataFrame({
    484     'feature': X_test.columns,
    485     'importance': pi.importances_mean
    486 }).sort_values('importance', ascending=False).head(20)
    488 # Visualization

File /databricks/python/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:203, in validate_params.<locals>.decorator.<locals>.wrapper(*args, **kwargs)
    200 to_ignore += ["self", "cls"]
    201 params = {k: v for k, v in params.arguments.items() if k not in to_ignore}
--> 203 validate_paramet

In [0]:
models_dict = {
    'RandomForest':     best_rf,
    'ExtraTrees':       best_et,
    'GradientBoosting': best_gb,
    'XGBoost':          best_xgb,
    'LightGBM':         best_lgb,
    'CatBoost':         best_cat,
}

# Permutation Importance
def resolve_model(name: str):
    if name == 'StackEnsemble':
        return stack
    if name == 'WeightedEnsemble':
        # Pick best individual (excluding WeightedEnsemble)
        best_individual = comp_df.drop('WeightedEnsemble').index[0]
        return resolve_model(best_individual)
    # otherwise a plain model from models_dict
    return models_dict[name]

importance_model = resolve_model(best_name)
importance_title = best_name if best_name != 'WeightedEnsemble' else f"{comp_df.drop('WeightedEnsemble').index[0]} (from Weighted)"

# Safety check
if importance_model is None:
    raise ValueError(f"Could not resolve model for importance: {best_name}")

print(f"\nCalculating permutation importance for {importance_title}...")
pi = permutation_importance(
    importance_model,
    X_test.iloc[:2000], y_test.iloc[:2000],
    n_repeats=10, random_state=42, n_jobs=-1
)

imp_df = (pd.DataFrame({'feature': X_test.columns, 'importance': pi.importances_mean})
            .sort_values('importance', ascending=False)
            .head(20))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top features
axes[0].barh(imp_df['feature'].iloc[::-1], imp_df['importance'].iloc[::-1])
axes[0].set_title(f'Top 20 Features — {importance_title}')
axes[0].set_xlabel('Permutation Importance')

# Predictions vs Actual
if best_name == 'WeightedEnsemble':
    test_preds = [m.predict(X_test) for m in models_dict.values()]
    test_pred = np.sum(optimal_weights * np.vstack(test_preds), axis=0)
else:
    test_pred = importance_model.predict(X_test)

axes[1].scatter(y_test, test_pred, alpha=0.5, s=20)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual CO(GT)')
axes[1].set_ylabel('Predicted CO(GT)')
axes[1].set_title(f'Predictions vs Actual — {best_name}')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [0]:
# Save XGBoost model

import shutil
import json
import pickle

print("\n" + "="*70)
print("SAVING XGBOOST MODEL")
print("="*70)

# Run ID
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="tags.mlflow.runName = 'bayes_XGBoost'",
    order_by=["start_time DESC"],
    max_results=1
)

if not runs:
    print("ERROR: XGBoost run not found")
else:
    run_id = runs[0].info.run_id
    model_uri = f"runs:/{run_id}/model"
    
    print(f"\nRun ID: {run_id}")
    print(f"Model URI: {model_uri}")
    
    # Get XGBoost metrics
    xgb_metrics = results.get('XGBoost', {})
    
    # Register with MLFlow Model Registry
    print("\n" + "-"*70)
    print("REGISTERING TO MODEL REGISTRY")
    print("-"*70)
    
    registered_model_name = "xgboost_final_best"
    
    try:
        model_version = mlflow.register_model(model_uri, registered_model_name)
        
        print(f"Model registered: {registered_model_name}")
        print(f"Version: {model_version.version}")
        
        description = f"""XGBoost model for CO(GT) air quality prediction

            Performance Metrics (Test Set):
            - RMSE: {xgb_metrics.get('rmse', 0):.4f}
            - MAE: {xgb_metrics.get('mae', 0):.4f}
            - R²: {xgb_metrics.get('r2', 0):.4f}
            - MAPE: {xgb_metrics.get('mape', 0):.2f}%
            - SMAPE: {xgb_metrics.get('smape', 0):.2f}%

            Model Details:
            - Algorithm: XGBoost with Bayesian optimization and early stopping
            - Features: {len(feature_cols)} engineered features
            - Hyperparameter Search: BayesSearchCV (50 iterations)
            - Target Transform: Log transformation
            - Dataset: UCI Air Quality Dataset

            Enhanced Features:
            - Extended lag features (1-72 hours)
            - Rolling statistics (3h to 1 week)
            - Rate of change features
            - Pollutant interaction features
            - Environmental interactions
            - Time-based categorical features
            """
        
        client.update_model_version(
            name=registered_model_name,
            version=model_version.version,
            description=description
        )
        
        print("Description updated")
        
        # Try to transition to Production
        try:
            client.transition_model_version_stage(
                name=registered_model_name,
                version=model_version.version,
                stage="Production",
                archive_existing_versions=True
            )
            print("Stage: Production")
        except Exception as stage_error:
            print(f"Stage transition skipped: {str(stage_error)[:50]}")
            print("Set stage manually in MLflow UI if needed")
        
        # Add tags
        try:
            client.set_model_version_tag(
                name=registered_model_name,
                version=model_version.version,
                key="model_type",
                value="XGBoost"
            )
            client.set_model_version_tag(
                name=registered_model_name,
                version=model_version.version,
                key="optimization",
                value="Bayesian"
            )
            print("Tags added")
        except:
            pass
            
    except Exception as reg_error:
        print(f"Registration error: {str(reg_error)[:100]}")
    
    # Save model offline
    print("\n" + "-"*70)
    print("EXPORTING MODEL OFFLINE")
    print("-"*70)
    
    # Create data directory if it doesn't exist
    os.makedirs("./data", exist_ok=True)
    
    # Download MLflow artifacts
    local_dir = "./data/xgboost_model"
    if os.path.exists(local_dir):
        shutil.rmtree(local_dir)
    
    # Create directory for download
    os.makedirs(local_dir, exist_ok=True)
    
    artifact_path = client.download_artifacts(run_id, "model", local_dir)
    print(f"Model downloaded to: {artifact_path}")
    
    # Create zip
    zip_base = "./data/xgboost_model"
    shutil.make_archive(zip_base, 'zip', local_dir)
    print(f"Zipped: {zip_base}.zip")
    
    # Save as pickle
    pickle_path = "./data/xgboost_model.pkl"
    with open(pickle_path, 'wb') as f:
        pickle.dump(best_xgb, f)
    print(f"Pickle: {pickle_path}")
    
    # Create features.json
    features_spec = {
        "model_name": "XGBoost Air Quality CO Prediction (Enhanced)",
        "target": target_col,
        "n_features": len(feature_cols),
        "feature_names": feature_cols,
        "feature_categories": {
            "lag_features": [f for f in feature_cols if 'lag' in f],
            "rolling_features": [f for f in feature_cols if 'rolling' in f],
            "rate_of_change": [f for f in feature_cols if 'diff' in f],
            "interactions": [f for f in feature_cols if any(x in f for x in ['_x_', '_ratio_'])],
            "temporal": [f for f in feature_cols if any(x in f for x in ['hour', 'day', 'month', 'sin', 'cos', 'weekend', 'rush', 'night', 'winter', 'summer'])]
        },
        "preprocessing": {
            "target_transform": "log1p",
            "feature_scaling": "none",
            "missing_values": "forward/backward interpolation"
        }
    }
    
    features_path = "./data/features.json"
    with open(features_path, 'w') as f:
        json.dump(features_spec, f, indent=2)
    print(f"Features spec: {features_path}")
    
    # Create model metadata
    metadata = {
        "model_info": {
            "name": registered_model_name if 'model_version' in locals() else "xgboost_final_best",
            "version": model_version.version if 'model_version' in locals() else "N/A",
            "run_id": run_id,
            "algorithm": "XGBoost with Bayesian Optimization"
        },
        "performance": {
            "test_rmse": float(xgb_metrics.get('rmse', 0)),
            "test_mae": float(xgb_metrics.get('mae', 0)),
            "test_r2": float(xgb_metrics.get('r2', 0)),
            "test_mape": float(xgb_metrics.get('mape', 0)),
            "test_smape": float(xgb_metrics.get('smape', 0)),
            "test_mase": float(xgb_metrics.get('mase', 0))
        },
        "dataset": {
            "train_samples": len(X_train),
            "valid_samples": len(X_valid),
            "test_samples": len(X_test),
            "total_features": len(feature_cols)
        },
        "usage": {
            "load_mlflow": 'mlflow.pyfunc.load_model("./data/xgboost_model/model")',
            "load_pickle": "pickle.load(open('./data/xgboost_model.pkl', 'rb'))",
            "predict": "predictions = model.predict(X)"
        }
    }
    
    metadata_path = "./data/model_metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Metadata: {metadata_path}")
    
    # Example input for testing
    example_input = X_test.iloc[0].to_dict()
    example_path = "./data/example_input.json"
    with open(example_path, 'w') as f:
        json.dump(example_input, f, indent=2)
    print(f"Example input: {example_path}")